## 실습 [9-1]  

### 1. 실습명 : N-gram 언어 모델로 문장 생성하기
### 2. 실습 목적 및 설명
 
*   파이썬의 NLTK 패키지를 이용하여 N-gram 언어 모델을 구축한다
*   네이버에서 오픈 소스로 제공하는 nsmc 영화 리뷰 데이터셋을 이용해 문장을 생성한다.

### 3. 관련 장(챕터) : 9.2.2 N-gram 언어 모델 (N-gram Language Model)
### 4. 코드

In [16]:
import nltk
from nltk.util import ngrams
from nltk import word_tokenize
from nltk import ConditionalFreqDist
from nltk.probability import ConditionalProbDist, MLEProbDist
import numpy as np
import codecs
from tqdm import tqdm
import random

In [17]:
!apt-get update

/bin/sh: apt-get: command not found


In [19]:
# 한국어 처리에 필요한 konlpy 패키지를 설치하기 전에 선행 파일을 설치한다. 
# !brew update


!brew install g++ openjdk-8-jdk python-dev python3-dev

# !pip3 install JPype1-py3

# !pip3 install konlpy

# !JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"

from konlpy.tag import Okt

Updating Homebrew...
==> Auto-updated Homebrew!
Updated 1 tap (homebrew/core).
==> Updated Formulae
exploitdb           media-info          openrct2            tflint

Error: No available formula with the name "g++" 
==> Searching for a previously deleted formula (in the last month)...
  git -C "$(brew --repo homebrew/core)" fetch --unshallow

Error: No previously deleted formula found.
==> Searching for similarly named formulae...
These similarly named formulae were found:
aacgain                                  grunt-completion
abnfgen                                  grv
adplug                                   gsar
ag                                       gsasl
agda                                     gsettings-desktop-schemas
agedu                                    gsl
aggregate                                gsmartcontrol
aircrack-ng                              gsoap
algernon                                 gspell
algol68g                                 gssdp
align           

Error: No formulae found in taps.


AttributeError: module '_jpype' has no attribute '_hasClass'

In [9]:
from konlpy.tag import Okt
# NLTK 패키지를 이용하여 입력 텍스트를 N-gram 형태로 변환한다.
sentence = "나는 매일 아침 지하철을 탄다"

AttributeError: module '_jpype' has no attribute '_hasClass'

In [6]:
# NLTK 사용을 위하여 선행 패키지를 설치한다.
nltk.download('punkt')

# 입력 텍스트를 띄어쓰기 기준으로 토큰화한다.
tokens = word_tokenize(sentence) 

[nltk_data] Downloading package punkt to /Users/edwardwon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
print(tokens)

['나는', '매일', '아침', '지하철을', '탄다']


In [8]:
# 한국어의 단어는 띄어쓰기를 기준으로 하지 않기 때문에 konlpy를 이용해 형태소를 기준으로 토큰화한다.
tagger = Okt()

def tokenize(text):
  tokens = ['/'.join(t) for t in tagger.pos(text)]
  return tokens

tokens = tokenize(sentence)
print(tokens)

NameError: name 'Okt' is not defined

In [ ]:
# 토큰을 N-gram의 형태로 바꾸어준다. 
# ngrams 함수의 두 번째 인자로 N값을 지정할 수 있다.
bigram = ngrams(tokens, 2)
trigram = ngrams(tokens, 3)

In [ ]:
# N-gram을 출력해본다.
print("bigram: ")
for b in bigram:
  print(b)

print("\ntrigram: ")
for t in trigram:
  print(t) 

In [ ]:
# padding을 통해 입력 데이터에 문장의 시작과 끝을 알리는 토큰을 추가한다. 
bigram = ngrams(tokens, 2, pad_left=True, pad_right=True, left_pad_symbol="<s>", right_pad_symbol="</s>")
print("bigrams with padding: ")
for b in bigram:
  print(b)

In [ ]:
# 문장 생성을 위하여 네이버 영화 리뷰 데이터셋을 다운로드한다.
%%time
!wget -nc -q https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

In [ ]:
# 다운로드 받은 데이터셋을 읽고 인덱스와 라벨을 제외한 텍스트 부분만 가져온다.
# codecs 패키지는 대용량 파일을 조금씩 읽을 수 있게 해준다. 

with codecs.open("ratings_train.txt", encoding='utf-8') as f:
  data = [line.split('\t') for line in f.read().splitlines()] # \n 제외
  data = data[1:] # header 제외
print("데이터셋: ", data[:10])
docs = [row[1] for row in data] # 텍스트 부분만 가져옴
print("\n텍스트 데이터:", docs[:5])
print("\n문장 개수: ",len(docs)) # 총 15만개의 문장으로 이루어진 데이터셋임을 알 수 있다.

In [ ]:
# 토큰화한 텍스트 데이터의 bigram을 모두 리스트에 추가한다.
sentences = []
for d in tqdm(docs):
  tokens = tokenize(d)
  bigram = ngrams(tokens, 2, pad_left=True, pad_right=True, left_pad_symbol="<s>", right_pad_symbol="</s>")
  sentences += [t for t in bigram]

In [ ]:
print(sentences[:10])

In [ ]:
cfd = ConditionalFreqDist(sentences)
print(cfd["<s>"].most_common(5))

In [ ]:
# 주어진 토큰(c) 다음에 가장 많이 등장하는 n개의 단어를 반환하는 함수를 만든다.
def most_common(c, n, pos=None):
  if pos is None:
    return cfd[tokenize(c)[0]].most_common(n)
  else:
    return cfd["/".join([c, pos])].most_common(n)

In [ ]:
print(most_common("나", 10))

In [ ]:
# 단어별 등장 빈도를 기반으로 조건부 확률을 추정한다.
cpd = ConditionalProbDist(cfd, MLEProbDist)

In [ ]:
# “.” 다음에 “</s>”가 올 확률을 출력한다.
print(cpd[tokenize(".")[0]].prob("</s>"))

In [ ]:
# 토큰 c 다음에 토큰 w가 bigram으로 함께 등장할 확률을 구한다.
def bigram_prob(c, w):
  context = tokenize(c)[0]
  word = tokenize(w)[0]
  return cpd[context].prob(word)

In [ ]:
print(bigram_prob("이", "영화"))

In [ ]:
print(bigram_prob("영화", "이"))

In [ ]:
# 조건부 확률을 알게 되면 가장 확률이 높은 토큰열을 토대로 문장을 생성할 수 있다.
def generate_sentence(seed=None, debug=False):
  if seed is not None:
    import random
    random.seed(seed)
  c = "<s>"
  sentence = []
  while True:
    if c not in cpd:
      break
    w = cpd[c].generate()

    if w == "</s>":
      break
    
    word = w.split("/")[0]
    pos = w.split("/")[1]

    # 조사, 어미 등을 제외하고 각 토큰은 띄어쓰기로 구분하여 생성한다.
    if c == "<s>":
      sentence.append(word.title())
    elif c in ["`", "\"","'","("]:
      sentence.append(word)
    elif word in ["'", ".", ",", ")", ":", ";", "?"]:
      sentence.append(word)
    elif pos in ["Josa", "Punctuation", "Suffix"]:
        sentence.append(word)
    elif w in ["임/Noun", "것/Noun", "는걸/Noun", "릴때/Noun",
                "되다/Verb", "이다/Verb", "하다/Verb", "이다/Adjective"]:
        sentence.append(word)
    else:
        sentence.append(" " + word)
    c = w

    if debug:
      print(w)

  return "".join(sentence)

In [ ]:
print(generate_sentence(2))

In [ ]:
generate_sentence(2, debug=True)

### 5. 결과
： 위 예제는 코퍼스 내의 등장 빈도에 기반하여 문장을 생성한다. bigram 언어 모델로 생성한 것이기 때문에 인접한 두 단어는 그나마 자연스럽지만 멀리 떨어진 단어와는 전혀 무관한 모습을 보인다. 또한 생성된 문장의 전체적인 문맥이 부자연스러우며 통사적으로 부적절한 모습도 보인다. 이는 코퍼스 내의 정보만으로 제한된 단어 조합만을 고려하는 N-gram 언어 모델의 한계로 보인다. 위 예제는 단순화를 위해 전처리와 규칙 처리를 최소화하였는데, 데이터셋을 늘리고 한국어 특징에 맞게 전처리를 진행한다면 보다 향상된 결과를 얻을 수 있을 것이다.


<참고>

> https://datascienceschool.net/view-notebook/a0c848e1e2d343d685e6077c35c4203b/


